In [1]:
import numpy as np
import time
from conv_nn import utils
from conv_nn.module import Sequential
from conv_nn.layer import Conv, batchnorm, ReLU, maxpool, Dropout, flatten, Linear
from conv_nn.loss import MultiLogisticLoss
from conv_nn.nn import ERMNeuralNetClassifier

In [2]:
print("Running main.py")
train_data = np.load('fmnist_train.npy', allow_pickle=True).item()
test_data = np.load('fmnist_test.npy', allow_pickle=True).item()

X = train_data['data']
y = train_data['labels']
X_test = test_data['data']

# Preprocessing X
X = X[..., np.newaxis] # add channel dimension
if X.max() > 1: X = X / 255.

X_test = X_test[..., np.newaxis] # add channel dimension
if X_test.max() > 1: X_test = X_test / 255.

# Split into Xfm_train, yfm_train, Xfm_val, yfm_val
Xfm_train, yfm_train, Xfm_val, yfm_val = utils.create_split(X, y, 0.8)

# Create model
model = Sequential([
    Conv(1, 32, 4, 2, 1), # 14 x 14 x 32
    ReLU(), 
    batchnorm(32), 
    Conv(32, 64, 4, 2, 2), # 8 x 8 x 64
    ReLU(),
    batchnorm(64),
    Conv(64, 128, 3, 1, 1), # 8 x 8 x 128
    maxpool(2, 2), # 4 x 4 x 128
    ReLU(),
    batchnorm(128),
    flatten(),
    Linear(4*4*128, 256),
    Dropout(0.5),
    Linear(256, 10)
])
loss = MultiLogisticLoss(k=10)
cnn_clf = ERMNeuralNetClassifier(model, loss)
sgd_kwargs = {
    'batch_size': 64,
    'n_epochs': 2,
    'eta': 0.01,
    'verbose': True, # Enable printing INSIDE SGD
    'verbose_epoch_interval': 1,
}

Running main.py


In [3]:
# Fit model
t = time.time()
cnn_clf.fit(Xfm_train[:100], yfm_train[:100], **sgd_kwargs)
print(f'Time taken to fit: {time.time() - t}')


Training complete, train_obj: 3.7289121594670744
Time taken to fit: 5.46673321723938


In [6]:
list = np.arange(0, 100)
for i in range(-(100//-35)):
    print(list[i*35:(i+1)*35])


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34]
[35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
 59 60 61 62 63 64 65 66 67 68 69]
[70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93
 94 95 96 97 98 99]


In [5]:
n_params = 0
for param in model.get_params():
    for p in param:
        n_params += p.size
print(f"Number of parameters: {n_params}")

Number of parameters: 653130


In [ ]:
# Evaluate model
print(f"Validation Error: {utils.empirical_err(yfm_val, cnn_clf.predict(Xfm_val))}")
y_test_preds = cnn_clf.predict(X_test)
fname = 'fmnist_test_pred.csv'
output = np.vstack((np.arange(y_test_preds.shape[0]), y_test_preds)).T
np.savetxt(fname, output, fmt="%d", delimiter=',', comments='', header='id,label')